Install packages and load data

In [1]:
import re
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import csv

pd.options.mode.chained_assignment = None  # default='warn'

# List of countries that do not have a break in the y axis
no_ED_countries = ["Argentina", "Brazil", "India", "Indonesia", "Malaysia", "Mauritius"]

Update data from gsheet

In [42]:
url = "https://docs.google.com/spreadsheets/d/1jLNfP3iuteUJrH0zS9qWONskyKh9pFcl1hKSlgEc-I8/gviz/tq?tqx=out:csv&sheet=All+data"
all_data = pd.read_csv(url)
all_data.to_csv("raw_df.csv", index=False)

Select countries (multiple possible)

In [3]:
def choose_country(Choose):
    Country = c.value

    print(str(Country))
c = widgets.SelectMultiple(
    options=list(all_data.country.unique()), 
    description='Country(ies):',
)
j = widgets.interactive(choose_country, Choose = c)
display(j)

interactive(children=(SelectMultiple(description='Country(ies):', options=('Argentina', 'Australia', 'Brazil',…

The following snippet updates 'bottom_chart.csv' for the selected country, and 'top_chart.csv' for any series with a minimum value that is greater than 100. These files determine the data that is displayed in the final chart.

In [40]:
c.observe(choose_country, 'value')
Country = list(c.value)

# load data
all_data = pd.read_csv("raw_df.csv")

for x in Country:
    country_data = all_data.loc[(all_data["country"] == x)]
    country_data = country_data[country_data["series_code"].str.contains("F")]
    country_data["col"] = country_data["measure"] + " - " + country_data["welfare_concept"]

    bottom_chart = pd.DataFrame()
    top_chart = pd.DataFrame()

    for i in country_data["series_code"].unique():
        final_series_df = country_data[country_data["series_code"] == i][["year", "value", "col"]]
        df_min = min(final_series_df["value"])
        final_series_df = final_series_df.rename(columns={"value": final_series_df["col"].unique()[0]}).drop(columns=["col"]).set_index('year')
        if df_min < 100:
            bottom_chart = pd.concat([bottom_chart, final_series_df], axis=1).sort_values("year")
        else:
            top_chart = pd.concat([top_chart, final_series_df], axis=1).sort_values("year")
    
    bottom_chart = bottom_chart.fillna("NA")
    bottom_chart.to_csv(x + "/bottom_chart.csv")
    top_chart = top_chart.fillna("NA")
    if not top_chart.empty:
        top_chart.to_csv(x +"/top_chart.csv")

    

The following snippet updates 'source_series_chart.js', which determines the data shown in the source series, and 'chart.html', which determines the series descriptions

In [41]:
c.observe(choose_country, 'value')
Country = list(c.value)

# load data
all_data = pd.read_csv("raw_df.csv")

all_data["source_legend"] = (
    all_data["short_reference"] + " - " + all_data["welfare_concept"]
)

for x in Country:
    # print(x)

    country_data = all_data.loc[(all_data["country"] == x)]

    final_series_list = list(
        country_data[
            (country_data["series_code"].str[0].isin(["F"]))
        ].series_code.unique()
    )

    data_list = []
    div_list = []

    # used for the colour of the subheadings
    colour_dict = {
        "Earnings Dispersion": "#104E8B",
        "Overall Income Inequality": "#008000",
        "Poverty": "#e31a1c",
        "Wealth Inequality": "#FF9912",
        "Top Income Shares": "#6a3d9a",
    }

    # iterate through list of final series
    i = 0
    while i < len(final_series_list):
        final_series = final_series_list[i]
        dimension = (
            country_data.loc[(all_data["series_code"] == final_series)]
            .dimension.unique()
            .item()
        )

        measure = (
            country_data.loc[(all_data["series_code"] == final_series)]
            .measure.unique()
            .item()
        )

        welfare_concept = (
            country_data.loc[(all_data["series_code"] == final_series)]
            .welfare_concept.unique()
            .item()
        )

        description = (
            country_data.loc[(all_data["series_code"] == final_series)]
            .description.unique()
            .item()
        )

        sources_used = (
            country_data.loc[(all_data["series_code"] == final_series)]
            .source_codes_used.unique()
            .item()
            .split(",")
        )

        preferred = str((
            country_data.loc[(all_data["series_code"] == final_series)]
            .preferred_definition.unique()
            .item()
        )).replace("*", "★").replace("nan", "")
        

        # construct json for final series
        final_series_df = country_data.loc[country_data["series_code"] == final_series]
        # for each final series, 'series' = 0 (required by chart js)
        final_series_df["series"] = 0
        final_series_df = final_series_df.rename(columns={"year": "x", "value": "y"})
        # assign max and min value for each series
        df_max = max(final_series_df["x"])
        df_min = min(final_series_df["x"])
        values = final_series_df[["x", "y", "series"]]
        # convert to dict
        values = values.to_dict("records")
        # create nested dict to fit required js format
        data = [{"key": "Chartbook series", "type": "line", "values": values, "yAxis": 1}]
        div = []

        # construct json for source series
        j = 0
        while j < len(sources_used):
            # create df of source series used
            source_series_df = country_data.loc[
                country_data["series_code"].isin(sources_used)
            ]
            # iterate through each source series
            source_series_df = source_series_df.loc[
                source_series_df["series_code"] == sources_used[j]
            ]
            # each source series has a unique 'series' value, starting from one (required by chart js)
            source_series_df["series"] = j + 1
            source_series_df = source_series_df.rename(columns={"year": "x", "value": "y"})
            values = source_series_df[["x", "y", "series"]]
            # add df_min and df_max of all values used to construct final series to each source series, in order to prevent zoom function
            add = pd.DataFrame(
                [[df_min, "null", j + 1], [df_max, "null", j + 1]],
                columns=["x", "y", "series"],
            )
            values = (
                pd.concat([values, add]).sort_values(by=["x"]).drop_duplicates(subset=["x"])
            )
            # convert to dict
            values = values.to_dict("records")
            # create nested dict and append
            data.append(
                {
                    "key": source_series_df["source_legend"].unique().item(),
                    "type": "scatter",
                    "values": values,
                    "yAxis": 1,
                }
            )
            j += 1
        # write string as it should appear in source_series_chart.js
        data = "$scope.data" + str(i) + " = " + str(data)
        data = data.replace("[", "[\n").replace("},", "},\n")

        # print(" " + source_series_df["dimension"].unique().item() + " - " + source_series_df["measure"].unique().item() + " - " + source_series_df["welfare_concept"].unique().item())

        # store string
        data_list.append(data)
        # create div of final series description as it should appear in chart.html
        div = (
            "<h2><span style='color:"
            + colour_dict[str(dimension)]
            + "'>"
            + dimension
            + ": </span>"
            + measure
            + " - "
            + welfare_concept 
            + " " 
            + preferred
            + "</h2><p><b>What is the source data that the Chartbook relies on? </b>"
            + description
            + "</p><h4>How does the Chartbook series compare with the source data?</h4>\n"
            + "<nvd3 options='options' data='data"
            + str(i)
            + "'></nvd3>"
        )
        # store div
        div_list.append(div)
        i += 1

    # reformat to js
    string = (
        str("; \n".join(data_list))
        .replace("'null'", "null")
        .replace("'key'", "key")
        .replace("'type'", "type")
        .replace("'values'", "values")
        .replace("'yAxis'", "yAxis")
    )

    div_string = str("\n").join(div_list)

    # edit source_series_charts.js
    with open(x + "/source_series_charts.js", "r") as file:
        old_script = file.read()
    new_script = re.sub("(?<=insert data\n)(.*\n.*)*", string + "\n});", old_script)
    open(x + "/source_series_charts.js", "w").write(new_script)

    # edit chart.html - specify ED or no_ED
    html = "chart_ED.html"
    if x in no_ED_countries:
        html = "chart_no_ED.html"

    with open(html, "r") as file:
        old_script = file.read()
    new_script = re.sub("(<!-- insert country -->\n).*", x, old_script)
    new_script = re.sub("(?<=<!-- insert data -->\n).*", div_string, new_script)
    open(x + "/chart.html", "w").write(new_script)

The following snippet updates the references in chart.html

In [31]:
references = """Argentina,https://docs.google.com/spreadsheets/d/1Dc8tx6cTio_AugopaxWkcUNKcp_bMEkwZZRmaZ-LkvQ/edit?usp=sharing
Australia,https://docs.google.com/spreadsheets/d/1_WIBAjDLO7ufWuBFhRFr-GMl6dLgLz47-uF5EvC48ZY/edit?usp=sharing
Brazil,https://docs.google.com/spreadsheets/d/1IaA-lvbRlixYMLy5nW6xxJolaZyO4DK_0XvLQuLjJJs/edit?usp=sharing
Canada,https://docs.google.com/spreadsheets/d/1zxuxAXriOrp0x_dxklVbULKGqG3fEOGBkouUNWx7AY8/edit?usp=sharing
Finland,https://docs.google.com/spreadsheets/d/1ZakjK-hP6s4tLJZCEFjR7NVVqTkb6AXwgpvfpSwsT-I/edit?usp=sharing
France,https://docs.google.com/spreadsheets/d/1sySmwtRs_MvRrcVj52o9b0nG-bzbPMu1haIoSK8IWqk/edit?usp=sharing
Germany,https://docs.google.com/spreadsheets/d/1g8gGUHRye1L7hEu3HqYgM56RvjHrc3LD85kBwRZcYj4/edit?usp=sharing
Iceland,https://docs.google.com/spreadsheets/d/1YP9WzHiFVPZLLZ91_bMNa-EzrCy9ME_5nmDUZu7SN4A/edit?usp=sharing
India,https://docs.google.com/spreadsheets/d/1hyXWzYakYa8NKYHVIvEqjAvV9YeVti4gOxJkT4UqSlI/edit?usp=sharing
Indonesia,https://docs.google.com/spreadsheets/d/1A6HH3eHNo2ubWGe6XisQFINtnGcE9d1WmCCHRYGSe8M/edit?usp=sharing
Italy,https://docs.google.com/spreadsheets/d/1ovdfVn5BJymTzItKbZsvyt6qDjPoKsVLST23lms_1qc/edit#gid=1830755962
Japan,https://docs.google.com/spreadsheets/d/1pwS0793rjAZNcVHf4EbrpIGXBGo8C74oP7p0A9H3HXA/edit?usp=sharing
Malaysia,https://docs.google.com/spreadsheets/d/1DOyjueTHGkWQ8-0skJ5c5_tO8x6y4mbUCPp5nhRR30I/edit?usp=sharing
Mauritius,https://docs.google.com/spreadsheets/d/1_T_B9cGEGdKX6zNsh3cIUg_dzUlpoJ98oSPOKg7ga7Q/edit?usp=sharing
Netherlands,https://docs.google.com/spreadsheets/d/1L_0W3MMKbvcAIOiFGZTkQOcar7JUkzgVgbH0RnMAkag/edit?usp=sharing
New Zealand,https://docs.google.com/spreadsheets/d/1-ncVJ4sUG8j_5ThnN3_T9wzWdbo26pr1rYy5xDJF0f4/edit#gid=1830755962
Norway,https://docs.google.com/spreadsheets/d/1SNMeX0RWQE-rTkj3rAxK6uZQmQaqa2pr2QFP5wW2kb0/edit#gid=1830755962
Portugal,https://docs.google.com/spreadsheets/d/1EdqTg2eDWQZOcuKY-LCNkNwOcrgIFVpl3MxVNIjYhLU/edit?usp=sharing
Singapore,https://docs.google.com/spreadsheets/d/1tkNPvqgSfkPdyvRR8GPLkYOVkzNesPKyPFXTiWhT2H0/edit?usp=sharing
South Africa,https://docs.google.com/spreadsheets/d/1DG9yxRwQ2QcCOiUU0_u-YqGOtALMNvBboywp9t6tl0I/edit?usp=sharing
Spain,https://docs.google.com/spreadsheets/d/1xXFMMP4glUrPevj4-rW04mqKQj8ebPAiP2tKNx8l6QA/edit?usp=sharing
Sweden,https://docs.google.com/spreadsheets/d/1Ylee87yl-XL1PMeK9LMgLK5NkIxe5cmb7PdC53AoMWM/edit?usp=sharing
Switzerland,https://docs.google.com/spreadsheets/d/1NXeIVqzbCAUGE6KpGLURIDVKUxXK8qBWOWfuo9Wn1Os/edit?usp=sharing
UK,https://docs.google.com/spreadsheets/d/1qa1LD_R9F3hdnVcbo-MB6mAMoBOG-IcnSypQXzCeMj0/edit?usp=sharing
US,https://docs.google.com/spreadsheets/d/1ONdnOFLa-SxtMe3uxtv1J3hZY16Li9mBf9Oi2gE4YwI/edit?usp=sharing"""

c.observe(choose_country, 'value')
Country = list(c.value)

lines = references.splitlines()
reader = csv.reader(lines)
references = list(reader)

ref_list = []

for x in Country:
    search = x
    for i in references:
        if i[0] == search:
            url = re.sub("\/edit.*", "/gviz/tq?tqx=out:csv&sheet=References", i[1])
            # print(url)
            refs = pd.read_csv(url)
            refs["Country"] = i[0]
            # print(refs)

            x = 0
            html_string = ""
            li = ""
            while x < len(refs):
                ref_list.append(refs.iloc[x])
                li = (
                    str("<li><a href = '")
                    + str(refs.iloc[x]["Reference URL"])
                    + str("'>")
                    + str(refs.iloc[x]["Reference"])
                    + str("</a></li>\n")
                )
                if not li:
                    break
                html_string += li
                x += 1

            html_string = re.sub("<a href = 'nan'>", "", html_string)
            html_string = "></div>\n<h2>References</h2>\n" + html_string + "</body>"

            with open(str(i[0]) + "/chart.html", "r") as file:
                old_script = file.read()
            new_script = re.sub("<\/nvd3>\n *<\/div>((.|\n)*)<\/body>", html_string, old_script)
            open(str(i[0]) + "/chart.html", "w").write(new_script)